In [ ]:
#Declan Halbert
#U79196431

# CS 237 Lab Three: Poker Probability
In this lab we will explore Poker Probability, which is calculating the probability of various hands in the game of poker. This is, again, exploring how to confirm our theoretical understanding with experiments. If our experiments, as we increase the number of trials, converge to our theoretical calculation, then we have almost certainly analyzed it correctly. 

There are many versions of poker (see <a href="http://www.wikihow.com/Play-Poker">here</a>) but the game we will study is called "five-card draw." It is <a href="https://www.pokernews.com/strategy/5-card-draw-rules-how-to-play-five-card-draw-poker-23741.htm">described</a> as follows:

<blockquote>Once everyone has paid the ante, each player receives five cards face down. A round of betting then occurs. If more than one player remains after that first round of betting, there follows a first round of drawing. Each active player specifies how many cards he or she wishes to discard and replace with new cards from the deck. If you are happy with your holding and do not want to draw any cards, you “stand pat.”


Once the drawing round is completed, there is another round of betting. After that if there is more than one player remaining, a showdown occurs in which the player with the best five-card poker hand wins.
</blockquote>

The only part we will care about is the final calculation of which hand wins: basically, the least probable hand wins.  When you learn poker, then, one of the first things you have to learn is the ordering of the hands from most to least likely. <i>Poker probability</i> refers to calculating the exact probabilities of hands. The <a href="https://en.wikipedia.org/wiki/Poker_probability">Wikipedia article</a> contains the exact results and the formulae used to calculate them. 

In this lab we will develop a framework for dealing 5-card hands and empirically estimating the probabilites of various hands. In fact, we will be able to do nearly all the hands commonly encountered. Our only constraint is that for the two rarest hands, the probability is so small it would take too long to get a reasonable estimate, and so we shall ignore them. 

This should complete your understanding of the counting techniques covered in lecture. 

In [1]:
# Here are some imports which will be used in code that we write for CS 237

# Jupyter notebook specific 

from IPython.display import Image
from IPython.core.display import HTML 
from IPython.display import display_html
from IPython.display import display
from IPython.display import Math
from IPython.display import Latex
from IPython.display import HTML   

# Imports potentially used for this lab

import numpy as np                # arrays and functions which operate on array
from numpy import linspace, arange
import matplotlib.pyplot as plt   # normal plotting
import seaborn as sns             # Fancy plotting 
import pandas as pd               # Data input and manipulation

from numpy.random import random, randint, uniform, choice, shuffle
from collections import Counter

%matplotlib inline


## Preface: Card Games and Probability
First we will first explore how to encode a standard deck of 52 playing cards, how to perform various tests on cards, and how to deal hands. To remind you, here is the illustration from hw 01 showing all the cards:
<a href="http://www.cs.bu.edu/fac/snyder/cs237/Homeworks/images/hw01.PlayingCards.png"> cards</a>.

In [279]:
# We will represent cards as a string, e.g., 'AC' will be Ace of Clubs

# Denominations: 'J' = Jack, 'Q' = Queen, 'K' = King, 'A' = Ace, 
Denominations = ['2','3','4','5','6','7','8','9','10','J','Q','K','A']

# Suits 'S' = Spades, 'H' = Hearts, 'D' = Diamonds, 'C' = Clubs  
Suits = ['C', 'H', 'S', 'D']

# Note that colors are determined by the suits (hearts and diamonds are red, others black,
# so, AC is Black
    
# List comprehensions are a great way to avoid explicit for loops when creating lists

Deck =  [(d+s) for d in Denominations for s in Suits]   # Note the double for loop

def Suits():
    deck2 = []
    for i in Deck: 
        if len(i) > 2:
            deck2.append(i[2])
            continue
        deck2.append(i[1])
    return deck2

FinalSuit = Suits()
    
print( FinalSuit )


['C', 'H', 'S', 'D', 'C', 'H', 'S', 'D', 'C', 'H', 'S', 'D', 'C', 'H', 'S', 'D', 'C', 'H', 'S', 'D', 'C', 'H', 'S', 'D', 'C', 'H', 'S', 'D', 'C', 'H', 'S', 'D', 'C', 'H', 'S', 'D', 'C', 'H', 'S', 'D', 'C', 'H', 'S', 'D', 'C', 'H', 'S', 'D', 'C', 'H', 'S', 'D']


In [280]:
# Now we can "deal" cards by choosing randomly from the deck

def dealCard():
    return choice(Deck)           # choice randomly chooses an element of a list

print( dealCard() )

3D


In [281]:
# When dealing a hand in cards, the selection of cards is without replacement, that is, cards are removed from
# the deck one by one and not put back. This can be simulated in the choice function by setting the replace 
# parameter to False. 

def dealHand():
    return choice(Deck, replace=False, size=5)

def dealHandSuit():
    return choice(FinalSuit, replace=False, size=5)  

print( dealHandSuit() )

['D' 'D' 'S' 'S' 'S']


In [282]:
# extract the denomination and the suit from a card

def denom(c):
    return c[0:-1]

def suit(c):
    return c[-1]

# The RANK of a denomination will be its position in the list 2, 3, ...., K, A. This will be used in an essential
# way in our code below. Although in the diagram linked above, Ace is below 2, the Ace is actually considered to be ordered
# above the King, for example in determining a straight. 

#  rank(2) = 0, ...., rank(10) = 8, rank(Jack) = 9, rank(Queen) = 10, rank(King) = 11, rank(Ace) = 12

def rank(c):
    return Denominations.index(denom(c))

# Now we want to identify various kinds of cards 

def isHeart(c):
    return ( suit(c) == 'H')

def isDiamond(c):
    return ( suit(c) == 'D')

def isClub(c):
    return ( suit(c) == 'C')

def isSpade(c):
    return ( suit(c) == 'S')

def isRed(c):
    return ( isHeart(c) or isDiamond(c) )

def isBlack(c):
    return (not isRed(c))

def isFaceCard(c):
    return rank(c) >= 9
    

## Example Problem: What is probability that a 5-card hand has exactly 3 red cards?
Remember that in finite probability, for any event A, $$P(A) = \frac{| A |}{|S|}.$$  Therefore, what we need to do in problems involving the probability of various kinds of hands in card games is to count the number of possible such hands, and divide by the total number of all possible hands. We develop analytical tools in lecture to do this, but here we are going to estimate it with repeated trials of dealing hands and testing for a given kind of hand. 

For the following code, notice carefully how list comprehensions are used. Run the code as given to see how it works and then try setting the num_trials to 10,000.

In general for all but the last problem, we will use 10,000 trials to get a reasonable estimate of the probability. Since 1/10000 = 0.0001 this means our resolution for experimental probabilities is 4 decimal places.  

In [283]:
# Return True iff the number of red cards in the hand h is 3
def threeRed(h):
    redCards = [c for c in h if isRed(c)]
    return (len(redCards) == 3)

# Run the experiment for 10,000 trials
# Print out probability that a 5-card hand has exactly 3 red cards

num_trials = 10000
trials = [dealHand() for k in range(num_trials)]       # create list of 10000 hands randomly dealt

if(num_trials <= 10):       # Just for this example, you don't need to do this unless you are debugging
    print(trials)
    
hands =  [threeRed(h) for h in trials]                 # convert this to list of true and false values

if(num_trials <= 10):
    print(hands)
    
prob = hands.count(True) / num_trials                  # count the number of True values and divide by num_trials

# probability for 10,000 trials should be close to analytical value of 0.3251

print('\nProbability of exactly 3 red cards in a 5-card hand is ' + str(prob))




Probability of exactly 3 red cards in a 5-card hand is 0.3265


In [284]:
# here is another way to do it, but it is a bit cryptic!

# trials was calculated above

prob2 = sum( [1 for h in trials if threeRed(h)] ) / num_trials

print('\nProbability of exactly 3 red cards in a 5-card hand is ' + str(prob2))


Probability of exactly 3 red cards in a 5-card hand is 0.3265


In [285]:
# If you like cryptic, then you can put it all, including the calculation of trials in one line! 
# Here are two examples. Note: these run the experiment again,  so they won't match the previous 2 answers precisely.

prob3 = sum( [1 for h in [dealHand() for k in range(num_trials)] if threeRed(h)] ) / num_trials

print('\nProbability of exactly 3 red cards in a 5-card hand is ' + str(prob3))

prob4 = sum( [1 for k in range(num_trials) if threeRed(dealHand())] ) / num_trials

print('\nProbability of exactly 3 red cards in a 5-card hand is ' + str(prob4))



Probability of exactly 3 red cards in a 5-card hand is 0.3148

Probability of exactly 3 red cards in a 5-card hand is 0.3204


## Problem 1: What is probability that a 5-card hand has at least 3 Diamonds?

In [286]:
# Run the experiment for 10,000 trials
# Print out probability that a 5-card hand has 3, 4, or 5 diamonds.

def threeDiamonds(h):
    Diamonds = [c for c in h if isDiamond(c)]
    return (len(Diamonds) >= 3)

num_trials = 10000
trials = [dealHand() for k in range(num_trials)] 

if(num_trials <= 10):       # Just for this example, you don't need to do this unless you are debugging
    print(trials)

hands =  [threeDiamonds(h) for h in trials]

if(num_trials <= 10):
    print(hands)
    
prob = hands.count(True) / num_trials 


print('\nProbability that a 5-card hand has at least 3 Diamonds: ' + str(prob))




Probability that a 5-card hand has at least 3 Diamonds: 0.0941


## Problem 2: What is probability of a flush in Poker?
In Poker, a <i>flush</i> is 5 cards of the same suit, but excludes straight flushes and royal flushes; these, however, are so rare, that they will not affect our approximation, so we just have to determine if all suits are the same. 

In [287]:
# Run the experiment for 10,000 trials
# Print out probability that a 5-card hand has all the same suit

# To verify correctness, see link above for poker frequencies of various hands

def Flush(h):
    count = 0
    hello = 0
    for i in h:
        if ord(h[0]) == ord(h[hello]):
            count += 1
        hello += 1
    if count == 5:
        return True
    return False    

num_trials = 10000
trials = [dealHandSuit() for k in range(num_trials)] 

if(num_trials <= 10):       # Just for this example, you don't need to do this unless you are debugging
    print(trials)

hands =  [Flush(h) for h in trials]

if(num_trials <= 10):
    print(hands)
    
prob = hands.count(True) / num_trials 


print('\nProbability that there is a Flush ' + str(prob))





Probability that there is a Flush 0.0022


## Problem 3: What is probability of a straight in Poker?
In poker, a <i>straight</i> a hand in which the ranks form a contiguous sequence, e.g., 2,3,4,5,6. The suits do not matter. As with flushes, we exclude straight flushes and royal flushes.  

In [288]:
# Print out probability that a 5-card hand is a straight
# Hint: Create a histogram of the rank frequencies using Counter: all frequencies should be 1 (each rank occurs
#   at most once), and then the min and max rank should be what distance apart??

def Straight(h):
    o = []
    for i in range(len(h)):
        o.append(rank(h[i]))
    if len(o) != len(set(o)):
        return False
    rank_range = max(o) - min(o) + 1
    if rank_range == 5 and len(o) == 5:
        return True
    return False

num_trials = 10000
trials = [dealHand() for k in range(num_trials)] 

if(num_trials <= 10):       # Just for this example, you don't need to do this unless you are debugging
    print(trials)

hands =  [Straight(h) for h in trials]

if(num_trials <= 10):
    print(hands)
    
prob = hands.count(True) / num_trials 


print('\nProbability of a straight: ' + str(prob))

# To verify correctness, see link above for poker frequencies of various hands


Probability of a straight: 0.0028


## Problem 4: Rank Signature of a poker hand

Let us define the <i>rank signature</i> of a hand as an ordered histogram of the ranks occurring in the hand; that is, we create a histogram of the ranks occurring in the hand, extract the values (ignoring the keys, i.e., the actual ranks), and order this sequence. Here are some examples:
 - No pair/ high card, five cards all of different ranks (e.g., Ace, 4, 2, King, 8): [1,1,1,1,1]
 - One pair, 2 cards of the same rank, and 3 more all of different ranks (e.g., 2,2,6,3,Ace): [1,1,1,2]
 - Two pair, 2 pairs (of different ranks) and one card of a different rank (e.g., 2,2,Ace,3,Ace): [1,2,2]
 - Full house, 2 cards of the same rank, and 3 cards of the same rank (e.g., 8,Jack,8,8,Jack): [2,3]
 
It is not important what suits are involved in these hands, and so they can be defined solely in terms of the ranks involved. The importance of this concept is that once we write a function to estimate the probability of a given signature, we can then immediately calculate the probability of many different poker hands.

For this problem you must write a function which calculate the probability that a 5-card hand has a given signature and verify it by calculating the probability of no pair / high card.

In [289]:
# Run the experiment for 10,000 trials
# Print out probability that a 5-card hand has a given signature
# Test using high card hand

def has_rank_signature(h,rank_signature):
    o = []
    
    for i in range(len(h)):
        o.append(rank(h[i]))
    fuckthisshit = Counter(o)
    
    final = []
    
    for j in range(5):
        if fuckthisshit[o[j]] != 0:
            final.append(fuckthisshit[o[j]])
        del fuckthisshit[o[j]]
    
    final.sort()
    
    if final == rank_signature:
        return True
    return False
    
    
    
    
    # Hint: create a dictionary of the hand and extract the values, and sort them, then compare with rank_signature
    return False

def probability_of_rank_signature(rank_signature,num_trials):
    return sum( [1 for h in [dealHand() for k in range(num_trials)] if has_rank_signature(h,rank_signature)] ) / num_trials

# probability should be close to analytical value of 0.501177

print('Probability of no pair / high card in poker is ' + str(probability_of_rank_signature([1,1,1,1,1],10000)))

Probability of no pair / high card in poker is 0.5129


## Problem Five: Using rank signature to calculate five different poker hands

For all of these except the last, you should do 10,000 trials. 

### Problem 5 (A): What is probability of one pair in Poker?

In [275]:
print('Probability of one pair in Poker ' + str(probability_of_rank_signature([1,1,1,2],10000)))

Probability of one pair in Poker 0.423


### Problem 5 (B): What is probability of two pairs in Poker?

In [274]:
print('Probability of two pairs in Poker ' + str(probability_of_rank_signature([1,2,2],10000)))

Probability of two pairs in Poker 0.049


### Problem 5 (C): What is probability of three of a kind in Poker?

In [270]:
print('Probability of three of a kind in Poker ' + str(probability_of_rank_signature([1,1,3],10000)))

Probability of three of a kind in Poker 0.0186


### Problem 5 (D): What is probability of a full house in Poker?


In [278]:
print('Probability of a full house in Poker ' + str(probability_of_rank_signature([2,3],10000)))

Probability of a full house in Poker 0.0013


### Problem 5 (E): What is probability of four of a kind in Poker?
The probability here is so small that you should run the experiment 1000,000 times, which will give you five digits of precision. 

In [277]:
print('Probability of four of a kind ' + str(probability_of_rank_signature([1,4],1000000)))

Probability of four of a kind 0.000225


### Challenge Problem (based on the example problem): 
Let p = the number of red cards in a hand. Without using the sampling technique presented in this lab, calculate 
the exact probability of p. Hint: think in terms of permutations. 